## Colormaps for Categorical Data

Colorcet primarily includes continuous colormaps, where each color is meant to be equally spaced in perceptual color space from the preceding and following colors. But for categorical data, it is important for each color to be visibly distinct from the other colors, _not_ nearby in color space, to make each category separately visible. 

There are [many sets of categorical colors available](https://graphicdesign.stackexchange.com/questions/3682), but these tend to be relatively small numbers of colors.  To support visualizing large numbers of categories, it would be good if an arbitrarily large number of colors could be chosen in a principled way from an available color space. Glasbey et al. presented such a method in:

> [Glasbey, Chris; van der Heijden, Gerie & Toh, Vivian F. K. et al. (2007)](https://strathprints.strath.ac.uk/30312/1/colorpaper_2006.pdf), "Colour displays for categorical images", Color Research & Application 32.4: 304-309.

Given a starting palette (e.g. white, black), this approach selects each new color to be maximally perceptually dissimilar from all the preceding colors, out of an allowed color space. Glasbey colors are available in [ImageJ](http://imagej.net/Glasbey) and for [R](https://github.com/btupper/catecolors), and there is a [Python implementation](https://github.com/taketwo/glasbey) of the method.  

Generating the colors from Python is time consuming, so we have generated some specific large sets of Glasbey colors to distribute in colorcet, as outlined below. We'll use [Bokeh](http://bokeh.pydata.org) with [HoloViews](http://holoviews.org) for displaying these colormaps so that you can zoom in for a closer look.

In [ ]:
import numpy as np
import colorcet as cc
import holoviews as hv
from holoviews import opts

hv.extension('bokeh')

### Testing distinctness

In order to test the viability of categorical color sets - we should display them on the color background that they will be used. This helps us discern whether the colors are distinguishable between themselves and against the background. We'll make a candy button style plot to capture this property:

In [ ]:
arr = np.arange(0, 100)
np.random.shuffle(arr)
zz = arr.reshape(10, 10)
xx, yy = np.meshgrid(np.arange(0,10), np.arange(0,10))

data = np.array([xx, yy, zz]).transpose().reshape(100, 3)

def candy_buttons(name, cmap=None, **kwargs):
    if cmap is None:
        cmap = cc.palette[name][:100]
        name = cc.get_aliases(name)
    options = opts.Points(color='color', size=35, tools=['hover'], 
                          yaxis=None, xaxis=None, height=450, width=450,
                          cmap=cmap, **kwargs)
    return hv.Points(data, vdims='color').opts(options).relabel(name)

## Color space

The results from the Glasbey algorithm depend on two factors: the initial starting set of colors, and the color space used. The default color space is the full [CIELAB](https://en.wikipedia.org/wiki/CIELAB_color_space) gamut, which results in colors of all the different available saturations, brightnesses, and hues:

In [ ]:
cc.colormaps("glasbey_bw")

If you need *n&nbsp;*colors that are distinct from each other and you have no other criteria to apply, just take the first *n&nbsp;*colors from this palette. Note that these colors won't necessarily be the _most_ distinct that are possible in a set of that size, because they are chosen iteratively (each compared to those that come before it) rather than being fully optimized for that specific size. The Glasbey algorithm can also be used to create such fully optimal sets, but it would not be practical to distribute all those sets and the color assignments would then vary each time a new data point was added, so this single set represents a good compromise.

### Light/Dark
What if you have additional constraints to consider?  For instance, if you are making a scatter or line plot on a black background, some of the colors above will show up vividly in contrast, and others will fade into the background because they are very dark. For use with dark or light backgrounds, we also provide Glasbey colors selected from a subset of the full CIELAB space:

In [ ]:
cc.colormaps("glasbey_bw_minc_20_maxl_70", "glasbey_bw_minc_20_minl_30")

Here we are using a naming scheme indicating the starting palette and any restrictions on the color space:
       
       glasbey_<starting_palette>[_<min|max>c_<chroma_value>][_<min|max>l_<lightness_value>][_hue_<start>_<end>]
       
The "light" colors were generated by eliminating all colors with lightness below 30% in the [CIECAM02-JCh](https://colorspacious.readthedocs.io/en/latest/tutorial.html#tutorial-perception) color space, and similarly the "dark" colors eliminated those with lightness above 70%. After the space was trimmed in that way, the same Glasbey procedure was followed as usual, making each subsequent color maximally distinct (from the remaining possible colors) from all those that precede it. These sets are intended to be useful on light and dark backgrounds, respectively.

In [ ]:
candy_buttons('glasbey_dark') + candy_buttons('glasbey_light', bgcolor='black')

### Cool/Warm
What if you are working within a particular color palette or want to combine two relatively large sets of categories? For this case we created a set of warm and cool colors from a subset of the full CIELAB space:

In [ ]:
cc.colormaps("glasbey_bw_minc_20_hue_330_100", "glasbey_bw_minc_20_hue_150_280")

Similar to how the dark and light sets were created, these warm and cool sets were generated by first trimming the colorspace and then applying the Glasbey procedure to make the colors maximally distincy. The "warm" colors were generated by only inlcuding colors with hues between 330° and 110°, and similarly the "cool" colors only include those between 150° and 280°. These sets can be used by themselves or in conjunction. It is important to note that by reducing the colorspace, we are decreasing the number of meaningfully distinct colors. So these sets should only be used when there is a strong reason to do so. 

In [ ]:
candy_buttons('glasbey_warm') + candy_buttons('glasbey_cool')

### No gray

We also generated a set where we eliminated gray values, i.e. those with chromaticity in the JCh color space below 20. These are useful if you truly need "colors", i.e. not grays: 

In [ ]:
cc.colormaps("glasbey_bw_minc_20")

You might have noticed that most of the colormaps mentioned above have more than one name. The short name is provided as an easy to remember alias for accessing the colormaps. In the case of the alias `glasbey` - we decided that the version with no gray would provide a better default than the regular glasbey set. In the plot below you can see that the "regular" glasbey (shown in the plot on the left), has severl colors that are very dark and hard to distinguish. In the "no gray" version on the right the colors seem more distinguishable. 

In [ ]:
candy_buttons('glasbey_bw') + candy_buttons('glasbey_bw_minc_20')

## Starting colors

For a given color space, the results also depend on the set of starting colors, which above was always [white, black]. We also removed those two colors from the final palettes, because white and black are generally already in use for plotting as the page background or for plot annotations.  (That said, if you do need the colormap to include white and black, just add `["white","black"]` to the front of one of the Bokeh palettes, as Bokeh palettes are simply Python lists of colors.)

In some cases, you may already have an initial set of colors that you use regularly, and you just want to extend that set to allow additional colors that are distinct from the original set. In that case, the Glasbey algorithm can be initialized with the starting set, and then extended using a selected color space.

Here, we've extended the Category10 colormap used in [bokeh](https://bokeh.pydata.org/en/latest/docs/reference/palettes.html#bokeh.palettes.d3) and originally from [D3](https://github.com/d3/d3-3.x-api-reference/blob/master/Ordinal-Scales.md#categorical-colors) to make a 256-color drop-in replacement that provides more colors for when they are needed, but the same first 10 colors:

In [ ]:
cc.colormaps("glasbey_category10")

To check how these compare with the original sets in Bokeh, we can render them side by side. Drag to the right or scroll out to see more of the Glasbey versions:

In [ ]:
from bokeh.palettes import Category10_10

(cc.colormap('bokeh: Category10', cmap=Category10_10, bounds=(0, 0, 10, 1)) +
 cc.colormap('glasbey_category10')).cols(1).redim.range(x=(0, 50))

Note that as you include more and more colors, it will be more and more difficult for people to perceive them as different, but this approach is still better than the alternative of having to cycle through the same colors again:

In [ ]:
candy_buttons('bokeh: Category10', cmap=Category10_10) + candy_buttons('glasbey_category10')

## Complete list, for reference

The colormaps with relatively short names above are expected to be the most useful in practice. The complete list of colormaps provided is:

In [ ]:
cc.colormaps(group='glasbey')